**Data** **cleaning**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os

CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
file_path = os.path.join(CURRENT_DIR,"raw" , "ObesityDataSet.csv")

# Charger le dataset
df = pd.read_csv(file_path)

# Afficher les premières lignes pour vérifier le chargement des données
print("Aperçu des données :")
print(df.head())

# Vérifier les valeurs manquantes
print("
Valeurs manquantes par colonne :")
print(df.isnull().sum())

# Vérifier si certaines valeurs sont codées différemment comme manquantes
print("
Valeurs potentiellement manquantes sous d'autres formes ('?', 'None', '') :")
print(df.isin(['?', 'None', '']).sum())

# Obtenir un résumé des données
print("
Résumé des colonnes et valeurs non nulles :")
print(df.info())

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Affichage des outliers avec un boxplot
numerical_cols = df.select_dtypes(include=['number']).columns
df_numerical = df[numerical_cols]

plt.figure(figsize=(14, 6))
sns.boxplot(data=df_numerical)
plt.title("Boxplots des variables numériques", fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Suppression des outliers
lower_weight, upper_weight = 40, 105
lower_age, upper_age = 10, 26

df_filtered = df[
    (df['Weight'] >= lower_weight) & (df['Weight'] <= upper_weight) &
    (df['Age'] >= lower_age) & (df['Age'] <= upper_age)
]

# Sauvegarde des données nettoyées
processed_path = os.path.join(CURRENT_DIR, "data", "processed", "dataset.csv")

# Créer le dossier si inexistant
os.makedirs(os.path.dirname(processed_path), exist_ok=True)
df_filtered.to_csv(processed_path, index=False)
print(f'✅ Données nettoyées sauvegardées dans : {processed_path}')

# Encodage de la colonne cible
le = LabelEncoder()
df_filtered['NObeyesdad'] = le.fit_transform(df_filtered['NObeyesdad'])

# Sauvegarde de l'encodeur
encoder_path = os.path.join(CURRENT_DIR, "data", "processed", "label_encoder.pkl")
with open(encoder_path, "wb") as file:
    pickle.dump(le, file)

print(f'✅ Label Encoder sauvegardé dans : {encoder_path}')



**Model TRaining**

In [ ]:
import joblib
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import gc
import psutil
import os
import pickle
import shap


# === data loading ===
# Get the absolute path of the current script (inside views/)
CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))

# Move up one level to reach the project root
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))

# Construct paths relative to the project root
DATA_PATH = os.path.join(BASE_DIR, "data","processed" , "dataset.csv")
df= pd.read_csv(DATA_PATH)

# === memory reducing ===
def optimize_dataframe(df):
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    return df

df = optimize_dataframe(df)
categorical_columns = ["Gender", "family_history_with_overweight", "NObeyesdad","FAVC","SMOKE","CAEC","SCC","CALC","MTRANS",]

# ==== spliting data ====
# Apply Label Encoding

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save encoders if you need to inverse transform later

# Split into features (X) and target (y)
X = df.drop("NObeyesdad", axis=1)  # Features
y = df["NObeyesdad"]  # Target (Obesity Level)


# Split into 80% Training and 20% Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training Set Size:", X_train.shape)
print("Testing Set Size:", X_test.shape)


# ==== FLAGS FOR SAMPLING METHODS ====
USE_SMOTE = True           # Enable/Disable Oversampling
USE_UNDERSAMPLING = True  # Enable/Disable Undersampling
USE_CLASS_WEIGHTS = False  # Set this to True to use class weighting


# Apply Oversampling (SMOTE)
if USE_SMOTE:
    smote = SMOTE(sampling_strategy="auto", random_state=42)  # 60% oversampling
    X_train, y_train = smote.fit_resample(X_train, y_train)
    print("Applied SMOTE Oversampling. New Training Set Size:", X_train.shape)

# Apply Undersampling (RandomUnderSampler)
if USE_UNDERSAMPLING:
    undersample = RandomUnderSampler(sampling_strategy="auto", random_state=42)  # 80% of majority class
    X_train, y_train = undersample.fit_resample(X_train, y_train)
    print("Applied Random UnderSampling. New Training Set Size:", X_train.shape)

# Compute Class Weights (If Selected)
class_weight_dict = None
if USE_CLASS_WEIGHTS:
    class_weights = compute_class_weight(class_weight="balanced",
                                         classes=np.unique(y_train), y=y_train)
    class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
    print("Applied Class Weights:", class_weight_dict)


# ===== training =====

# Initialize Model with Selected Class Weights
model = RandomForestClassifier(n_estimators=100, random_state=42,
                               class_weight=class_weight_dict if USE_CLASS_WEIGHTS else "balanced")
# Train Model
model.fit(X_train, y_train)


# Save the trained model to a file
with open("obesity_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model saved successfully!")


# Train SHAP explainer after training your model
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)  # Ensure X_test matches training format


# Save SHAP explainer to a file
with open("shap_explainer.pkl", "wb") as file:
      pickle.dump(explainer, file)

print("SHAP explainer saved successfully as shap_explainer.pkl!")


# === Memmory Optimization ===

def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # Convertir en Mo

memory_used = get_memory_usage()
print(f" Memory use after execution : {memory_used:.2f} Mo")
variables_a_supprimer = [var for var in globals().keys() if var not in ["get_memory_usage", "gc", "psutil", "pickle", "shap", "__name__", "__file__", "__builtins__"]]


for var in variables_a_supprimer:
    del globals()[var]

gc.collect()
print(" Memory freed !")


**Model** **Evaluation**

In [ ]:
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import gc
import psutil

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split

# === DATA & MODEL LOADING ===
CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))

# Construct paths
DATA_PATH = os.path.join(BASE_DIR, "data", "processed", "dataset.csv")
MODEL_PATH = os.path.join(BASE_DIR, "models", "obesity_model.pkl")

# Load dataset
df = pd.read_csv(DATA_PATH)

# Load trained model
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found: {MODEL_PATH}")

print(f"Loading model from: {MODEL_PATH}")
model = joblib.load(MODEL_PATH)

# === DATA PREPARATION ===
categorical_columns = ["Gender", "family_history_with_overweight", "FAVC", "SMOKE",
                       "CAEC", "SCC", "CALC", "MTRANS", "NObeyesdad"]

# Apply Label Encoding
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save encoders for inverse transform if needed

# Split features & target variable
X = df.drop("NObeyesdad", axis=1)
y = df["NObeyesdad"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === PREDICTION ===
y_pred = model.predict(X_test)

# === MODEL EVALUATION ===
accuracy = accuracy_score(y_test, y_pred)
print(f" Accuracy: {accuracy:.2f}")

print("\n Classification Report:\n", classification_report(y_test, y_pred))

# === CONFUSION MATRIX ===
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=set(y_test), yticklabels=set(y_test))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# === ROC-AUC SCORE ===
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))
roc_auc = roc_auc_score(y_test_binarized, model.predict_proba(X_test), multi_class="ovr")
print(f" ROC-AUC Score: {roc_auc:.4f}")

# === MEMORY OPTIMIZATION ===
def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # Convert to MB

memory_used = get_memory_usage()
print(f" Memory usage after execution: {memory_used:.2f} MB")

# Clean up memory
variables_to_keep = {"get_memory_usage", "gc", "psutil", "__name__", "__file__", "__builtins__"}
for var in list(globals().keys()):
    if var not in variables_to_keep:
        del globals()[var]

gc.collect()
print(" Memory freed!")


**Shap** **Explainer**

In [ ]:
import joblib
import shap
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === DATA AND MODEL LOADING ===
CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))
DATA_PATH = os.path.join(BASE_DIR, "data", "processed", "dataset.csv")
MODEL_PATH = os.path.join(BASE_DIR, "models", "obesity_model.pkl")

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found: {MODEL_PATH}")

print(f"🔹 Loading model from: {MODEL_PATH}")
model = joblib.load(MODEL_PATH)

# === DATA PREPARATION ===
df = pd.read_csv(DATA_PATH)

categorical_columns = ["Gender", "family_history_with_overweight", "FAVC", "SMOKE",
                       "CAEC", "SCC", "CALC", "MTRANS"]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save encoders for inverse transform if needed

X = df.drop("NObeyesdad", axis=1)
y = df["NObeyesdad"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# === SHAP EXPLAINER ===
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# === Ploting Shap ===
shap.summary_plot(shap_values, X_test)

print(" SHAP analysis completed successfully!")
